In [49]:
from dotenv import load_dotenv
import os

load_dotenv()

username = os.getenv('LFC_USERNAME')
password = os.getenv('LFC_PASSWORD')

In [55]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import re
import time

driver = webdriver.Chrome()

driver.get("https://chinese.littlefox.com/en")

driver.find_element(By.NAME, 'loginid').send_keys(username)
driver.find_element(By.NAME, 'loginpw').send_keys(password)
driver.find_element(By.CLASS_NAME, 'btn_login').click()
time.sleep(3)

content_list_url = "https://chinese.littlefox.com/en/story/contents_list/DP000798"
driver.get(content_list_url)

# Wait for the parent element to be present
list_wrap = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, 'list_wrap'))
)

# Find the thumb_wrap elements within the list_wrap
thumb_wraps = list_wrap.find_elements(By.CLASS_NAME, 'thumb_wrap')

# Loop through each element and click on it
for i in range(len(thumb_wraps)):
    try:
        # Refetch the thumb_wrap elements
        list_wrap = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'list_wrap'))
        )
        thumb_wraps = list_wrap.find_elements(By.CLASS_NAME, 'thumb_wrap')
        
        # Wait for the specific element to be clickable
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable(thumb_wraps[i])
        ).click()
        time.sleep(5)  # Adjust the sleep time as needed to allow for page load or other actions
        driver.get("https://chinese.littlefox.com/en/player_h5/view")
        
        # Extract the page source
        page_source = driver.page_source
        
        # Use regex to find the stream.m3u8 URL
        match = re.search(r'video_url":"(\\/contents_5\\/cn\\/hls\\/1080\\/[^"]+\\/stream\.m3u8\?_[^"]+)"', page_source)
        if match:
            stream_url = match.group(1).replace('\\/', '/')
            print(f"Found stream URL: {stream_url}")
        else:
            print("Stream URL not found")
        
        # Navigate back to the content list page
        driver.get(content_list_url)
        time.sleep(3)  # Adjust the sleep time as needed to allow for page load or other actions
    except StaleElementReferenceException:
        print("StaleElementReferenceException encountered. Refetching elements.")
        driver.get(content_list_url)
        time.sleep(3)

driver.quit()

Found stream URL: /contents_5/cn/hls/1080/2f09edcb8b/1600669648f6a4f71e72dfe084f2d4b5bf96963e02/stream.m3u8?_=1600669650
Found stream URL: /contents_5/cn/hls/1080/725902eac4/16006699598cbe9ce23f42628c98f80fa0fac8b19a/stream.m3u8?_=1600669960
